In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_json('uniswap_wbtc_usdc_swaps.json')
df.head()

,timestamp,datetime,blockNumber,txHash,sender,recipient,amount0,amount1,price,tick
0,2021-05-05 21:42:13,2021-05-05 21:42:13+00:00,12376730,0x9bf327df628323d51256f64e63e26acc15559776f646...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xe5C27478cDDF77670b16352D6B5752F60a96c71f,0.004221,-240.633362,57175,63490
1,2021-05-05 21:51:06,2021-05-05 21:51:06+00:00,12376776,0xd751b357b27be36d0b98faeaf9e1d078117ff3215f79...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xE3c14271E10b41fdF603c845434493884a8dB4d9,0.007017,-400.000000,57174,63490
2,2021-05-05 21:57:19,2021-05-05 21:57:19+00:00,12376802,0xccbe191bb29b7c89d941bd3f4de81aeb8a57c9e253a0...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x00dbc6E6b3420615f595FCB9DCD0D4a0e4514201,0.087727,-5000.000000,57158,63487
3,2021-05-05 21:59:00,2021-05-05 21:59:00+00:00,12376811,0x518fefce14365f42f5144da5503700e285540eade990...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xe623031eC40DAA3EF5f87b408a95c9221035291f,0.060222,-3431.560270,57148,63485
4,2021-05-05 22:08:57,2021-05-05 22:08:57+00:00,12376869,0xefd3b31c172ff21b8987ba4b224110f9e22dc3d9c726...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xE592427A0AEce92De3Edee1F18E0157C05861564,0.030485,-1736.872329,57142,63484


In [15]:
df.columns

Index(['timestamp', 'datetime', 'blockNumber', 'txHash', 'sender', 'recipient',
       'amount0', 'amount1', 'price', 'tick'],
      dtype='object')

In [16]:
df = df.drop(columns=['txHash', 'sender', 'recipient'])
df.head(5)

,timestamp,datetime,blockNumber,amount0,amount1,price,tick
0,2021-05-05 21:42:13,2021-05-05 21:42:13+00:00,12376730,0.004221,-240.633362,57175,63490
1,2021-05-05 21:51:06,2021-05-05 21:51:06+00:00,12376776,0.007017,-400.000000,57174,63490
2,2021-05-05 21:57:19,2021-05-05 21:57:19+00:00,12376802,0.087727,-5000.000000,57158,63487
3,2021-05-05 21:59:00,2021-05-05 21:59:00+00:00,12376811,0.060222,-3431.560270,57148,63485
4,2021-05-05 22:08:57,2021-05-05 22:08:57+00:00,12376869,0.030485,-1736.872329,57142,63484


In [17]:
# Group by 1-minute bins
ohlcv = df.resample('1T', on='datetime').agg({
    'price': ['first', 'max', 'min', 'last'],
    'amount1': lambda x: x.abs().sum()  # volume in token1 terms
})

/tmp/ipykernel_2993/1794400515.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlcv = df.resample('1T', on='datetime').agg({


In [18]:
ohlcv.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
ohlcv = ohlcv.dropna().reset_index()

ohlcv.head(5)

,datetime,Open,High,Low,Close,Volume
0,2021-05-05 21:42:00+00:00,57175.0,57175.0,57175.0,57175.0,240.633362
1,2021-05-05 21:51:00+00:00,57174.0,57174.0,57174.0,57174.0,400.000000
2,2021-05-05 21:57:00+00:00,57158.0,57158.0,57158.0,57158.0,5000.000000
3,2021-05-05 21:59:00+00:00,57148.0,57148.0,57148.0,57148.0,3431.560270
4,2021-05-05 22:08:00+00:00,57142.0,57142.0,57142.0,57142.0,1736.872329


In [19]:
ohlcv.tail(5)

,datetime,Open,High,Low,Close,Volume
177,2021-05-07 10:04:00+00:00,56408.0,56408.0,56408.0,56408.0,49900.649780
178,2021-05-07 10:13:00+00:00,56482.0,56482.0,56482.0,56482.0,24418.528386
179,2021-05-07 10:15:00+00:00,56481.0,56481.0,56481.0,56481.0,500.000000
180,2021-05-07 10:20:00+00:00,56480.0,56480.0,56480.0,56480.0,335.729448
181,2021-05-07 10:36:00+00:00,56501.0,56501.0,56501.0,56501.0,6938.522407


In [20]:
# 1hour resampling
ohlcv_hourly = df.resample('1H', on='datetime').agg({
    'price': ['first', 'max', 'min', 'last'],
    'amount1': lambda x: x.abs().sum()  # volume in token1 terms
})
ohlcv_hourly.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
ohlcv_hourly = ohlcv_hourly.dropna().reset_index()
ohlcv_hourly.head(5)

/tmp/ipykernel_2993/1751666748.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ohlcv_hourly = df.resample('1H', on='datetime').agg({


,datetime,Open,High,Low,Close,Volume
0,2021-05-05 21:00:00+00:00,57175,57175,57148,57148,9072.193632
1,2021-05-05 22:00:00+00:00,57142,57432,57001,57432,186212.516222
2,2021-05-05 23:00:00+00:00,57379,57379,57367,57367,20940.168315
3,2021-05-06 00:00:00+00:00,57349,57349,57215,57215,48825.795682
4,2021-05-06 01:00:00+00:00,57038,57038,57008,57014,68658.735417
